In [2]:
:dep polars = { version="0.24.3", features = ["algo","polars-algo","dtype-struct","dtype-time", "ipc", "ipc_streaming", "fmt", "lazy","list", "list_eval", "dot_product", "cum_agg", "list_to_struct", "cumulative_eval", "temporal", "dynamic_groupby"]}
:dep glob = { version = "0.3.0"}
:dep color-eyre = {version = "0.6.2"}
:dep plotly = { version = "0.8" }


In [4]:
use polars::io::ipc::{IpcReader, IpcStreamReader};
use polars::io::SerReader;
use polars::prelude::*;
use std::fs::File;
use color_eyre::{Result};

In [5]:
let pattern = "/home/leigh/projects/printnanny-gst-plugin/.tmp/fixture_0*.ipc";
let paths = glob::glob(&pattern).expect("Failed to parse glob pattern");

let lazyframes: Vec<LazyFrame> = paths
    .map(|p| {
        let p = p.unwrap();
        let f = File::open(&p).expect("file not found");
        IpcStreamReader::new(f).finish().unwrap().lazy()
    })
    .collect();
let boxdf = concat(&lazyframes, true, true).unwrap().collect().unwrap();
boxdf

shape: (11320, 7)
┌────────────┬────────────┬────────────┬──────────┬─────┬───────────┬──────────────────┐
│ detection_ ┆ detection_ ┆ detection_ ┆ detectio ┆ det ┆ detection ┆ ts               │
│ boxes_x0   ┆ boxes_y0   ┆ boxes_x1   ┆ n_boxes_ ┆ ect ┆ _scores   ┆ ---              │
│ ---        ┆ ---        ┆ ---        ┆ y1       ┆ ion ┆ ---       ┆ i64              │
│ f32        ┆ f32        ┆ f32        ┆ ---      ┆ _cl ┆ f32       ┆                  │
│            ┆            ┆            ┆ f32      ┆ ass ┆           ┆                  │
│            ┆            ┆            ┆          ┆ es  ┆           ┆                  │
│            ┆            ┆            ┆          ┆ --- ┆           ┆                  │
│            ┆            ┆            ┆          ┆ i32 ┆           ┆                  │
╞════════════╪════════════╪════════════╪══════════╪═════╪═══════════╪══════════════════╡
│ 0.457292   ┆ 0.590198   ┆ 0.482011   ┆ 0.614641 ┆ 0   ┆ 0.3828125 ┆ 2149652932087183 │
├╌╌

In [6]:
let score_threshold = 0.5;
let score_filter = col("detection_scores").gt(score_threshold);

boxdf.clone().lazy().filter(score_filter).collect()

Ok(shape: (817, 7)
┌────────────┬────────────┬────────────┬──────────┬─────┬───────────┬──────────────────┐
│ detection_ ┆ detection_ ┆ detection_ ┆ detectio ┆ det ┆ detection ┆ ts               │
│ boxes_x0   ┆ boxes_y0   ┆ boxes_x1   ┆ n_boxes_ ┆ ect ┆ _scores   ┆ ---              │
│ ---        ┆ ---        ┆ ---        ┆ y1       ┆ ion ┆ ---       ┆ u64              │
│ f32        ┆ f32        ┆ f32        ┆ ---      ┆ _cl ┆ f32       ┆                  │
│            ┆            ┆            ┆ f32      ┆ ass ┆           ┆                  │
│            ┆            ┆            ┆          ┆ es  ┆           ┆                  │
│            ┆            ┆            ┆          ┆ --- ┆           ┆                  │
│            ┆            ┆            ┆          ┆ i32 ┆           ┆                  │
╞════════════╪════════════╪════════════╪══════════╪═════╪═══════════╪══════════════════╡
│ 0.518654   ┆ 0.451672   ┆ 0.558763   ┆ 0.484425 ┆ 0   ┆ 0.75      ┆ 2146389279008721 │
├╌

In [80]:
// nozzle detections @ 15fps windowed by 1s interval/period

let every = "1s";
let period = "1s"; 
let offset = "0s";
let score_threshold = 0.5;
let ddof = 0; // delta degrees of freedom, used for std deviation / variance calculations. divisor = N - ddof, where N is the number of elements in set.

let group_options = DynamicGroupOptions{
    index_column: "ts".to_string(),
    every: Duration::parse(every),
    period: Duration::parse(period), 
    offset: Duration::parse(offset), 
    closed_window: ClosedWindow::Left,
    truncate: false,
    include_boundaries: true
};

let windowed_nozzle_df = boxdf.clone().lazy()
    .filter(col("detection_classes").eq(0))
    .filter(col("detection_scores").gt(score_threshold))
    .sort("ts", SortOptions { descending: false, nulls_last: false })
    .groupby_dynamic([col("detection_classes")], group_options )
    .agg([
        col("detection_scores").filter(col("detection_classes").eq(0)).count().alias("nozzle__count"),
        col("detection_scores").filter(col("detection_classes").eq(0)).mean().alias("nozzle__mean"),
        col("detection_scores").filter(col("detection_classes").eq(0)).std(ddof).alias("nozzle__std"),

        col("detection_scores").filter(col("detection_classes").eq(1)).count().alias("adhesion__count"),
        col("detection_scores").filter(col("detection_classes").eq(1)).mean().alias("adhesion__mean"),
        col("detection_scores").filter(col("detection_classes").eq(1)).std(ddof).alias("adhesion__std"),

        col("detection_scores").filter(col("detection_classes").eq(2)).count().alias("spaghetti__count"),
        col("detection_scores").filter(col("detection_classes").eq(2)).mean().alias("spaghetti__mean"),
        col("detection_scores").filter(col("detection_classes").eq(2)).std(ddof).alias("spaghetti__std"),

        col("detection_scores").filter(col("detection_classes").eq(3)).count().alias("print__count"),     
        col("detection_scores").filter(col("detection_classes").eq(3)).mean().alias("print__mean"),
        col("detection_scores").filter(col("detection_classes").eq(3)).std(ddof).alias("print__std"),

        col("detection_scores").filter(col("detection_classes").eq(4)).count().alias("raft__count"),     
        col("detection_scores").filter(col("detection_classes").eq(4)).mean().alias("raft__mean"),
        col("detection_scores").filter(col("detection_classes").eq(4)).std(ddof).alias("raft__std"),

    ])
    .collect()?;
windowed_nozzle_df

shape: (61, 19)
┌────────────┬────────────┬────────────┬─────────┬─────┬──────────┬───────────┬──────────┬─────────┐
│ detection_ ┆ _lower_bou ┆ _upper_bou ┆ ts      ┆ ... ┆ print__s ┆ raft__cou ┆ raft__me ┆ raft__s │
│ classes    ┆ ndary      ┆ ndary      ┆ ---     ┆     ┆ td       ┆ nt        ┆ an       ┆ td      │
│ ---        ┆ ---        ┆ ---        ┆ i64     ┆     ┆ ---      ┆ ---       ┆ ---      ┆ ---     │
│ i32        ┆ i64        ┆ i64        ┆         ┆     ┆ f32      ┆ u32       ┆ f32      ┆ f32     │
╞════════════╪════════════╪════════════╪═════════╪═════╪══════════╪═══════════╪══════════╪═════════╡
│ 0          ┆ 2149653000 ┆ 2149654000 ┆ 2149653 ┆ ... ┆ null     ┆ 0         ┆ null     ┆ null    │
│            ┆ 000000     ┆ 000000     ┆ 4389398 ┆     ┆          ┆           ┆          ┆         │
│            ┆            ┆            ┆ 15      ┆     ┆          ┆           ┆          ┆         │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌

In [51]:
extern crate plotly;
use plotly::common::{
    ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode, Title,
};
use plotly::layout::{Axis, BarMode, Layout, Legend, TicksDirection};
use plotly::{Bar, Plot, Scatter};

let series = windowed_nozzle_df.select_series(["ts", "detection_scores"])?;
let x: Vec<i64> = series[0].i64()?.into_no_null_iter().collect(); 
let y = series[1].f32()?.into_no_null_iter().collect();
let trace1 = Scatter::new(x, y);

let mut plot = Plot::new();
plot.add_trace(trace1);
plot.show()

()